In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [3]:
#Objetivo: Obtener los 10 jugadores que han aparecido en más partidos.

apariciones = spark.read.option('header','True').csv('../../data/practica/appearances.csv')
jugadores = spark.read.option('header','True').csv('../../data/practica/players.csv')

In [4]:
apariciones.printSchema()

root
 |-- player_id: string (nullable = true)
 |-- game_id: string (nullable = true)
 |-- appearance_id: string (nullable = true)
 |-- competition_id: string (nullable = true)
 |-- player_club_id: string (nullable = true)
 |-- goals: string (nullable = true)
 |-- assists: string (nullable = true)
 |-- minutes_played: string (nullable = true)
 |-- yellow_cards: string (nullable = true)
 |-- red_cards: string (nullable = true)



In [18]:
jugadores.show(5)

+---------+---------------+-------------------+-------------------+----------------+----------------------+-------------+----------+--------------+-----+------------+-------------------+---------------------------+--------------------+
|player_id|current_club_id|               name|        pretty_name|country_of_birth|country_of_citizenship|date_of_birth|  position|  sub_position| foot|height_in_cm|market_value_in_gbp|highest_market_value_in_gbp|                 url|
+---------+---------------+-------------------+-------------------+----------------+----------------------+-------------+----------+--------------+-----+------------+-------------------+---------------------------+--------------------+
|    38790|          28095|     dmitri-golubov|     Dmitri Golubov|           UdSSR|                Russia|   1985-06-24|    Attack|Centre-Forward| Both|         178|               NULL|                   675000.0|https://www.trans...|
|   106539|          28095| aleksandr-vasiljev| Aleksand

In [30]:
from pyspark.sql.functions import col,desc,lit
a = apariciones.alias('a')
j = jugadores.alias('j')

df_join = j.join(a, a.player_id == j.player_id,'inner')

In [13]:
#Objetivo: Obtener los 10 jugadores que han aparecido en más partidos.
df_join.groupBy(j.name).count().orderBy(desc('count')).limit(10).show()

+-----------+-----+
|       name|count|
+-----------+-----+
|   paulinho|  744|
|    marcelo|  688|
|     danilo|  648|
|  guilherme|  591|
| joao-pedro|  451|
|     felipe|  416|
|    charles|  412|
|luis-suarez|  393|
|scott-brown|  388|
|    rodrigo|  380|
+-----------+-----+



In [14]:
# Ejercicio 2: Promedio de edad por club
clubs = spark.read.option('header','True').csv('../../data/practica/clubs.csv')


In [16]:
clubs.printSchema()

root
 |-- club_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- pretty_name: string (nullable = true)
 |-- domestic_competition_id: string (nullable = true)
 |-- total_market_value: string (nullable = true)
 |-- squad_size: string (nullable = true)
 |-- average_age: string (nullable = true)
 |-- foreigners_number: string (nullable = true)
 |-- foreigners_percentage: string (nullable = true)
 |-- national_team_players: string (nullable = true)
 |-- stadium_name: string (nullable = true)
 |-- stadium_seats: string (nullable = true)
 |-- net_transfer_record: string (nullable = true)
 |-- coach_name: string (nullable = true)
 |-- url: string (nullable = true)



In [54]:
#2: Promedio de edad por club
from datetime import date
from pyspark.sql.functions import lit, datediff, floor,avg

c = clubs.alias('c')

df_p2 = j.join(c,j.current_club_id == c.club_id)

hoy = date.today()

df_final = df_p2.withColumn('Fecha_act',lit(hoy))

df_p2_edad = df_final.withColumn('edad',(floor(datediff(col('Fecha_act'),col('date_of_birth'))/ 365)))

In [48]:
df_p2_edad.show(5)

+---------+---------------+-------------------+-------------------+----------------+----------------------+-------------+----------+--------------+-----+------------+-------------------+---------------------------+--------------------+-------+------------+------------+-----------------------+------------------+----------+-----------+-----------------+---------------------+---------------------+--------------------+-------------+-------------------+----------------+--------------------+----------+----+
|player_id|current_club_id|               name|        pretty_name|country_of_birth|country_of_citizenship|date_of_birth|  position|  sub_position| foot|height_in_cm|market_value_in_gbp|highest_market_value_in_gbp|                 url|club_id|        name| pretty_name|domestic_competition_id|total_market_value|squad_size|average_age|foreigners_number|foreigners_percentage|national_team_players|        stadium_name|stadium_seats|net_transfer_record|      coach_name|                 url| Fe

In [ ]:
df_p2_edad.groupBy(c.name) \
    .agg(avg(col("edad")).alias("edad_promedio")) \
    .orderBy("edad_promedio", ascending=False) \
    .show(5)

+--------------------+------------------+
|                name|     edad_promedio|
+--------------------+------------------+
|    metalurg-donetsk| 38.57142857142857|
|thonon-evian-gran...|              38.5|
|         fc-penafiel| 38.27777777777778|
|      torpedo-moskau|             37.75|
|           cesena-fc|37.388888888888886|
+--------------------+------------------+
only showing top 5 rows



: 

In [ ]:
#🧠 Ejercicio 3: Goles por liga
